<a href="https://colab.research.google.com/github/SzymonNowakowski/Machine-Learning-2024/blob/master/Lab14-putting-it-all-together.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lab 14 - Putting It All Together


### Author: Szymon Nowakowski


# Introduction
---------------

This is our last class this semester. It responds to a request made by several students to explain the **design process** behind a complete neural network architecture.

This is, frankly, a difficult task — because such a process relies heavily on **intuition and experience**, neither of which can be fully explained. They are acquired through *“dupogodziny”* — the hours you spend working, trying, failing, adjusting, and learning.

I once attended a drawing school I liked a lot, and their motto was striking:

**Talent nie istnieje** — *Talent does not exist.*

The owner used to explain this quite eloquently. Every artist's output quality can be described by their **personal Gaussian distribution**. Talent is responsible only for the initial placement of the expected value of that distribution — sure, some people naturally produce better work *on average*, when they are young and untrained. But the most important factor determining the final quality of the art is **time spent practicing** — the *dupogodziny* you put in every day.

With that in mind, in this final class, I'll walk you through the **design and training** process of a neural network I most recently worked on:  
a **regressor for predicting period length in chloroplast grana**.

But the final and most important thought I woudl like to convey is that you need to spend your time practicing.

This is why the homework for the 13th class will in fact put you through your own design process of trial and error. Sure, I provided some suggestions, but I am also sure that you will need to work out what works for you and what not.


